# Script to map external files to SalishSeaCast grid and create the respective netcdf file

## Importing

In [1]:
import xarray as xr
import numpy as np
import os
from tqdm import tqdm
from salishsea_tools import grid_tools as gt


## Loading datasets

In [2]:
# Mask loading
mesh = xr.open_dataset('/home/sallen/MEOPAR/grid/mesh_mask202108.nc')
mask = mesh.tmask.to_numpy()
mask = mask[0,0]

# Daily dimensions loading
ds0 = xr.open_dataset('/data/ibougoudis/MOAD/files/integrated_original.nc')

# Obtaining lat and lon
lat = mesh.nav_lat
lon = mesh.nav_lon

lat = np.broadcast_to(lat,(len(ds0.time_counter),len(ds0.y),len(ds0.x)))
lon = np.broadcast_to(lon,(len(ds0.time_counter),len(ds0.y),len(ds0.x)))


## Calculation of the variables

In [3]:
def calculation(k):

    for j in tqdm(folders): 

        # Test for the proper date matching
        test = xr.open_dataset(path+j)
        if test.time_counter.dt.date[12] != ds0.time_counter[k].dt.date:
            print (ds0.time_counter[k].dt.date)

        inter, grid = gt.build_matrix(weight, (path+j))

        for i in range(0,24):

            solar_hourly[i] = gt.use_matrix(path+j, inter, grid, 'solar', i)
            u_hourly[i] =  gt.use_matrix(path+j, inter, grid, 'u_wind', i)
            v_hourly[i] =  gt.use_matrix(path+j, inter, grid, 'v_wind', i)
            temp_hourly[i] = gt.use_matrix(path+j, inter, grid, 'tair', i)
    
            solar2_hourly[i] = gt.use_matrix(path+j, inter, grid, 'therm_rad', i)
            pressure_hourly[i] =  gt.use_matrix(path+j, inter, grid, 'atmpres', i)
            precip_hourly[i] = gt.use_matrix(path+j, inter, grid, 'precip', i)
            humid_hourly[i] = gt.use_matrix(path+j, inter, grid, 'qair', i)

        solar[k] = solar_hourly.sum('hour').where(mask==1)
        wind_speed[k] = np.sqrt(u_hourly**2 + v_hourly**2).mean('hour').where(mask==1)
        temp[k] = temp_hourly.mean('hour').where(mask==1)
        
        solar2[k] = solar2_hourly.sum('hour').where(mask==1)
        pressure[k] = pressure_hourly.mean('hour').where(mask==1)
        precip[k] = precip_hourly.mean('hour').where(mask==1)
        humid[k] = humid_hourly.mean('hour').where(mask==1)
        
        k = k + 1
    
    return k


## File creation

In [4]:
def file_creation(variable, name):

    temp = variable.to_dataset(name=name)
    temp.to_netcdf(path='/data/ibougoudis/MOAD/external_inputs.nc', mode='a', encoding={name:{"zlib": True, "complevel": 9}})
    

## Creation of datasets

In [5]:
# Coordinates
coords_hourly = dict(hour=np.arange(0,24), y=ds0.y, x=ds0.x)
coords = dict(time_counter= ds0.time_counter, y=ds0.y, x=ds0.x)

# k is the general counter, j the daily one, i the hourly one
k = 0

# Hourly arrays
solar_hourly = xr.DataArray(coords=coords_hourly, dims = ['hour', 'y', 'x'])
u_hourly = xr.DataArray(coords=coords_hourly, dims = ['hour', 'y', 'x'])
v_hourly = xr.DataArray(coords=coords_hourly, dims = ['hour', 'y', 'x'])
temp_hourly = xr.DataArray(coords=coords_hourly, dims = ['hour', 'y', 'x']) 

solar2_hourly = xr.DataArray(coords=coords_hourly, dims = ['hour', 'y', 'x'])
pressure_hourly = xr.DataArray(coords=coords_hourly, dims = ['hour', 'y', 'x'])
precip_hourly = xr.DataArray(coords=coords_hourly, dims = ['hour', 'y', 'x'])
humid_hourly = xr.DataArray(coords=coords_hourly, dims = ['hour', 'y', 'x'])

# Daily arrays
solar = xr.DataArray(coords=coords, dims = ['time_counter', 'y', 'x'])
wind_speed = xr.DataArray(coords=coords, dims = ['time_counter', 'y', 'x'])
temp = xr.DataArray(coords=coords, dims = ['time_counter', 'y', 'x'])

solar2 = xr.DataArray(coords=coords, dims = ['time_counter', 'y', 'x'])
pressure = xr.DataArray(coords=coords, dims = ['time_counter', 'y', 'x'])
precip = xr.DataArray(coords=coords, dims = ['time_counter', 'y', 'x'])
humid = xr.DataArray(coords=coords, dims = ['time_counter', 'y', 'x'])

lat = xr.DataArray(data=lat,coords=coords, dims = ['time_counter', 'y', 'x'])
lon = xr.DataArray(data=lon,coords=coords, dims = ['time_counter', 'y', 'x'])


## Calculations

In [ ]:
# Four loops, one for each case

# First loop (15 Feb 2007 - 30 Apr 2011, different mask)
path = ('/results/forcing/atmospheric/GEM2.5/gemlam/')
folders = [x for x in os.listdir(path) if ((x[13:15]=='03' or x[13:15]=='04' or (x[13:15]=='02' and x[16:18] > '14')) and (x[8:12] < '2012'))]
folders.sort()
folders = [i for i in folders if i[18]=='.'] # Removing four double files
weight = ('/home/sallen/MEOPAR/grid/weights-gem2.5-gemlam_201702_pre22sep11.nc')
k = calculation(k)

# Second loop (15 Feb 2012 - 30 Apr 2014, different mask)
path = ('/results/forcing/atmospheric/GEM2.5/gemlam/')
folders = [x for x in os.listdir(path) if ((x[13:15]=='03' or x[13:15]=='04' or (x[13:15]=='02' and x[16:18] > '14')) and (x[8:12] > '2011'))]
folders.sort()
weight = ('/home/sallen/MEOPAR/grid/weights-gem2.5-gemlam_201702_22sep11onward.nc')
k = calculation(k)

# Third loop (15 Feb 2015 - 22 Feb 2023)
path = ('/results/forcing/atmospheric/GEM2.5/operational/')
folders = [x for x in os.listdir(path) if ((x[10:12]=='03' or x[10:12]=='04' or (x[10:12]=='02' and x[13:15] > '14')))]
folders.sort()
folders = [i for i in folders if i[15]=='.'] # Removing one double file
weight = ('/home/sallen/MEOPAR/grid/weights-gem2.5-ops.nc') # Created two files, one with each mask for the operational files time period
k = calculation(k)

# Fourth loop (23 Feb 2023 - 30 Apr 2024)
path = ('/results/forcing/atmospheric/continental2.5/nemo_forcing/')
folders = [x for x in os.listdir(path) if ((x[12:14]=='03' or x[12:14]=='04' or (x[12:14]=='02' and x[15:17] > '14')) and (x[7:11]<'2025'))]
folders.sort()
folders = folders[2:] # Cut the first two days in the folder
weight = ('/home/sallen/MEOPAR/grid/weights-continental2.5-hrdps_202108_23feb23onward.nc')
k = calculation(k)


## Calling the file creation

In [7]:
file_creation(solar, 'Summation_of_solar_radiation')
file_creation(wind_speed, 'Mean_wind_speed')
file_creation(temp, 'Mean_air_temperature')

file_creation(solar2, 'Summation_of_longwave_radiation')
file_creation(pressure, 'Mean_pressure')
file_creation(precip, 'Mean_precipitation')
file_creation(humid, 'Mean_specific_humidity')
file_creation(lat, 'Latitude')
file_creation(lon, 'Longitude')
